In [1]:
import dill as pickle
import aesara.tensor as aet
import pandas as pd
import pycmtensor as cmt
from pycmtensor.pycmtensor import PyCMTensorModel, Beta, Weights
from pycmtensor.functions import logit, neg_loglikelihood
from pycmtensor.optimizers import Adam
from pycmtensor.results import Results, Predict

swissmetro = pd.read_csv("data/swissmetro.dat", sep="\t")
db = cmt.Database("swissmetro", swissmetro, choiceVar="CHOICE")
globals().update(db.variables)
# Removing some observations
exclude = ((PURPOSE != 1) * (PURPOSE != 3) + (CHOICE == 0)) > 0
db.remove(exclude)

# additional steps to format database
db.data["CHOICE"] -= 1 # set the first choice to 0
db.choices = sorted(db.data["CHOICE"].unique()) # save original choices
db.autoscale(variables=['TRAIN_CO', 'TRAIN_TT', 'CAR_CO', 'CAR_TT', 
    'SM_CO', 'SM_TT'], default=100., verbose=False)

In [2]:
from aesara.tensor.random.utils import RandomStream

class ResLogitLayer:
    def __init__(self, input, w_in, w_out, activation_in=None, activation_out=None):
        assert w_in.shape[1].eval() == w_out.shape[0].eval()
        if isinstance(input, (list, tuple)):
            assert len(input) == w_in.shape[0].eval(), f"index.0 of w_in must be of the same length as input"
            input = aet.stacklists(input).flatten(2)

        assert isinstance(w_in, (Weights)), "w_in must be of type Weights"
        assert isinstance(w_out, (Weights)), "w_out must be of type Weights"
        self.w_in = w_in()
        self.w_out = w_out()
        if activation_in == None:
            activation_in = aet.sigmoid
        if activation_out == None:
            activation_out = aet.sigmoid
            
        h = activation_in(aet.dot(input.T, self.w_in))
        output = activation_out(aet.dot(h, self.w_out)).T
        self.input = input
        self.weights = [self.w_in, self.w_out]
        self.output = output + input

    
    def __repr__(self):
        return f"ResLogitLayer([{self.w_in.shape.eval()}, {self.w_out.shape.eval()}])"


class MNLmodel(PyCMTensorModel):
    def __init__(self, db):
        super().__init__(db)
        self.name = "myModel"
        self.inputs = db.tensors()  # keep track of inputs

        # declare model params here
        b_cost = Beta("b_cost", 0.0, None, None, 0)
        b_time = Beta("b_time", 0.0, None, None, 0)
        asc_train = Beta("asc_train", 0.0, None, None, 0)
        asc_car = Beta("asc_car", 0.0, None, None, 0)
        asc_sm = Beta("asc_sm", 0.0, None, None, 1)

        # b_time_s = Beta("b_time_s", 0.0, None, None, 0)

        # W1 = Weights("ResNet_01a", (3, 10), 0, True)
        # W2 = Weights("ResNet_01b", (10, 3), 0, True)

        # append model params to self.params list
        self.append_to_params(locals())
        # self.append_to_params([b_cost, b_time, asc_train, asc_car, asc_sm, W1, W2])

        # Definition of the utility functions
        # srng = RandomStream(seed=234)
        # rv_n = srng.normal(0, 1, size=(20,))
        # b_time_rnd = b_time + b_time_s * rv_n
        U_1 = b_cost * db["TRAIN_CO"] + b_time * db["TRAIN_TT"] + asc_train
        U_2 = b_cost * db["SM_CO"] + b_time * db["SM_TT"] + asc_sm
        U_3 = b_cost * db["CAR_CO"] + b_time * db["CAR_TT"] + asc_car
        U = [U_1, U_2, U_3]
        # rh = ResLogitLayer(U, W1, W2)

        # definition of the choice output
        self.y = db["CHOICE"]

        # symbolic expression for the choice model
        self.p_y_given_x = logit(U, [db["TRAIN_AV"], db["SM_AV"], db["CAR_AV"]])
        # self.p_y_given_x = logit(rh.output, [db["TRAIN_AV"], db["SM_AV"], db["CAR_AV"]])

        # declare Regularizers here:
        # L1 regularization cost
        self.L1 = abs(b_cost()) + abs(b_time())

        # L2 regularization cost
        self.L2 = b_cost() ** 2 + b_time() ** 2

        # symbolic expression for the cost fuction
        self.cost = neg_loglikelihood(self.p_y_given_x, self.y)
        self.cost = self.cost

        # symbolic description of how to compute prediction as class whose
        # probability is maximal
        self.pred = aet.argmax(self.p_y_given_x, axis=0)

In [3]:
# train function
model = cmt.train(MNLmodel, database=db, optimizer=Adam, batch_size=128, lr_init=0.01, max_epoch=999, notebook=True)

with open("myModel.pkl", "rb") as f:
    model = pickle.load(f)

result = Results(model, db, show_weights=True)

Building model...
dataset: swissmetro (6768)
batch size: 128
batches per epoch: 52
validation frequency: 52

Training model...


Loglikelihood:  -6964.662979  Score: 0.134

Epoch    0/51948:   0%|          | 0.00/51.9k [00:00<?, ?it/s]

Maximum Patience reached. Early stopping...
Optimization complete with accuracy of 62.544%
 with maximum loglikelihood reached @ epoch 366.
Results for model myModel
Number of Beta parameters: 4
Sample size: 6768
Init loglikelihood: -6964.663
Final loglikelihood: -5590.573
Likelihood ratio test: 2748.180
Accuracy: 62.544%
Rho square: 0.197
Rho bar square: 0.197
Akaike Information Criterion: 11189.15
Bayesian Information Criterion: 11216.43
Final gradient norm: 0.161

              Value   Std err     t-test   p-value Rob. Std err Rob. t-test Rob. p-value
asc_car    0.077562  0.042261   1.835315  0.066459     0.004942   15.695852          0.0
asc_sm          0.0         -          -         -            -           -            -
asc_train -0.561478  0.054498 -10.302725       0.0     0.049399  -11.366126          0.0
b_cost     0.013695  0.002798   4.894167  0.000001     0.007932    1.726466     0.084264
b_time     -1.17696  0.054507 -21.592746       0.0     0.021839  -53.891781        

In [4]:
from pycmtensor.results import Predict
from pycmtensor.pycmtensor import PyCMTensorModel
import dill as pickle

class MNLmodel(PyCMTensorModel):
    def __init__(self, db):
        super().__init__(db)

with open("myModel.pkl", "rb") as f:
    model = pickle.load(f)

Predict(model, db).choices()

,CHOICE
0,1
1,1
2,1
3,1
4,1
...,...
6763,1
6764,1
6765,1
6766,1


In [5]:
from tqdm import tqdm, trange
from time import sleep

pbar = tqdm(total=100)
for i in range(10):
    sleep(0.1)
    pbar.update(10)
pbar.close()

pbar.format_dict

100%|██████████| 100/100 [00:01<00:00, 90.74it/s]


{'n': 100,
 'total': 100,
 'elapsed': 1.102992057800293,
 'ncols': None,
 'nrows': None,
 'prefix': '',
 'ascii': False,
 'unit': 'it',
 'unit_scale': False,
 'rate': None,
 'bar_format': None,
 'postfix': None,
 'unit_divisor': 1000,
 'initial': 0,
 'colour': None}